In [8]:
# !pip install pandas
# !pip install numpy
import pandas as pd

df = pd.read_csv("~/Documents/Machine_Learning_101/2_Supervised_Learning_Linear_Regression/PPR-2018-Dublin.csv")
print list(df.columns.values)



['Date of Sale (dd/mm/yyyy)', 'Address', 'Postal Code', 'County', ' Price ', 'Not Full Market Price', 'VAT Exclusive', 'Description of Property', 'Property Size Description']


In [28]:
print df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18600 entries, 0 to 18599
Data columns (total 10 columns):
Date of Sale (dd/mm/yyyy)    18600 non-null object
Address                      18600 non-null object
Postal Code                  11667 non-null object
County                       18600 non-null object
 Price                       18600 non-null object
Not Full Market Price        18600 non-null object
VAT Exclusive                18600 non-null object
Description of Property      18600 non-null object
Property Size Description    4195 non-null object
Month                        18600 non-null int64
dtypes: int64(1), object(9)
memory usage: 1.4+ MB
None


In [32]:
print df[:5]

  Date of Sale (dd/mm/yyyy)                                   Address  \
0                02/01/2018                 1 ABBEY ST, HOWTH, DUBLIN   
1                02/01/2018         1 ROSEVILLE, LOWER ROAD, SHANKILL   
2                02/01/2018            19 ULVERTON RD, DALKEY, DUBLIN   
3                02/01/2018        236 PHIBSBORO RD, DUBLIN 7, DUBLIN   
4                02/01/2018  3 COLLEGE COURT, PORTRANE ROAD, DONABATE   

  Postal Code  County      Price  Not Full Market Price VAT Exclusive  \
0    Dublin 7  Dublin   710000.00                    No            No   
1   Dublin 18  Dublin   627230.70                    No            No   
2         NaN  Dublin  1130000.00                    No            No   
3    Dublin 7  Dublin   277000.00                    No            No   
4         NaN  Dublin   348017.62                    No            No   

                 Description of Property Property Size Description  Month  
0  Second-Hand Dwelling house /Apartment      

In [35]:
# data source: https://www.cso.ie/px/pxeirestat/Statire/SelectVarVal/saveselections.asp
cpi = {'Month': [1,2,3,4,5,6,7,8,9,10,11,12], 
    'CPI': [99.7,100.6,100.9,100.7,101.3,101.4,101.8,102.1,101.7,101.6,101.1,101.1]}
cpi = pd.DataFrame(data=cpi)
cpi[:5]

,CPI,Month
0,99.7,1
1,100.6,2
2,100.9,3
3,100.7,4
4,101.3,5


In [39]:
# df['Month']=pd.DatetimeIndex(df['Date of Sale (dd/mm/yyyy)']).month
# print(df['Month'].unique())
# df[' Price '] = df[' Price '].str.replace(',','')
df['Price'] =df[' Price '] .astype(float)
propertyPrice= df.groupby('Month')['Price'].mean()
print propertyPrice



Month
1     419429.178301
2     445577.873436
3     412006.294752
4     420850.732893
5     435558.075076
6     657038.695139
7     517144.403954
8     458760.895526
9     438277.919718
10    541269.692280
11    411043.681969
12    460954.650263
Name: Price, dtype: float64


In [40]:
cleanData=pd.merge(propertyPrice, cpi, left_on='Month', right_on='Month')
cleanData

,Month,Price,CPI
0,1,419429.178301,99.7
1,2,445577.873436,100.6
2,3,412006.294752,100.9
3,4,420850.732893,100.7
4,5,435558.075076,101.3
5,6,657038.695139,101.4
6,7,517144.403954,101.8
7,8,458760.895526,102.1
8,9,438277.919718,101.7
9,10,541269.692280,101.6


In [42]:
# sudo -H pip install astropy --ignore-installed sklearn
from sklearn import linear_model
import matplotlib.pyplot as plt
import numpy as np
X = np.array(cleanData[['CPI']])
Y = np.array(cleanData[['Price']])
plt.rc('font',family='STXihei',size=15)
plt.scatter(X,Y,60,color='blue',marker='o',linewidths=3,alpha=0.4)
plt.xlabel('CPI')
plt.ylabel('PropertyPrice')
plt.title('Data Information')
plt.show()

/System/Library/Frameworks/Python.framework/Versions/2.7/Extras/lib/python/matplotlib/collections.py:548: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == 'face':


In [43]:
clf = linear_model.LinearRegression()
clf.fit(X,Y)
print(clf.coef_,clf.intercept_)
print(clf.score(X,Y))

(array([[44330.23004811]]), array([-4016582.26542443]))
0.1579830604990079


There are two competing concerns: with less training data, your parameter estimates have greater variance. With less testing data, your performance statistic will have greater variance. Broadly speaking you should be concerned with dividing data such that neither variance is too high, which is more to do with the absolute number of instances in each category rather than the percentage.

If you have a total of 100 instances, you're probably stuck with cross validation as no single split is going to give you satisfactory variance in your estimates. If you have 100,000 instances, it doesn't really matter whether you choose an 80:20 split or a 90:10 split (indeed you may choose to use less training data if your method is particularly computationally intensive).

Assuming you have enough data to do proper held-out test data (rather than cross-validation), the following is an instructive way to get a handle on variances:

Split your data into training and testing (80/20 is indeed a good starting point)
Split the training data into training and validation (again, 80/20 is a fair split).
Subsample random selections of your training data, train the classifier with this, and record the performance on the validation set
Try a series of runs with different amounts of training data: randomly sample 20% of it, say, 10 times and observe performance on the validation data, then do the same with 40%, 60%, 80%. You should see both greater performance with more data, but also lower variance across the different random samples
To get a handle on variance due to the size of test data, perform the same procedure in reverse. Train on all of your training data, then randomly sample a percentage of your validation data a number of times, and observe performance. You should now find that the mean performance on small samples of your validation data is roughly the same as the performance on all the validation data, but the variance is much higher with smaller numbers of test samples


Last year, I followed Prof: Andrew Ng’s online machine learning course. His recommendation was

Training: 60%

Cross validation: 20%

Testing: 20%